## Big Data Mining
### Project 2
### 2019-2020

Kyriakos   Zormpas  
Alexandros Zotos  
Theodora  Koutsothanasi  
Evangelia Santorinaiou  dsc19024  

## Libraries

In [23]:
#!pip install namegenerator

In [3]:
import pandas as pd
import numpy as np

import namegenerator
from datetime import datetime, timedelta
import random
import time
import names

# Dataset Creation

Our scenario in order to build our dataset is that we have the restaurants from all over the world, name, id, when they were founded and their age, their coordinates, their speciality and their reviews. 

In [4]:
def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))

def random_date(start, end, prop):
    return str_time_prop(start, end, '%m/%d/%Y', prop)

To find the random latitude and longitude we used worldcitiespop.csv and we kept the country and the city that match each combination. We did not use AccentCity column because it is the same with City but with accents thus it not provides additional information. Also we removed Population because most of the values were Nan and Region because we will not use it in our scenario.

In [136]:
def get_latitude_longititude(df_lat):
    df_lat = pd.read_csv('worldcitiespop.csv')[:30000]
    df_lat = df_lat.loc[:,['Country', 'City', 'Latitude', 'Longitude']]

In [137]:
get_latitude_longititude(df_lat)
df_lat.head(5)

,Country,City,Latitude,Longitude
0,ad,aixas,42.483333,1.466667
1,ad,aixirivali,42.466667,1.500000
2,ad,aixirivall,42.466667,1.500000
3,ad,aixirvall,42.466667,1.500000
4,ad,aixovall,42.466667,1.483333


### Restaurants

In [97]:
restaurant_df = pd.DataFrame(columns=['id', 'name', 'cid', 'sid', 'founded', 'age'])


for i in range(df_lat.shape[0]):
    r_date = random_date("01/01/2008", "12/31/2019", random.random())
    
    restaurant_df.loc[i] = [np.random.randint(100000, 999999), 
                            namegenerator.gen(), 
                            np.random.randint(10000, 99999), 
                            np.random.randint(100000, 999999),  
                            r_date, 
                            int(str(datetime.today())[:4]) - int(r_date[6:])]
restaurant_df.head(5)

,id,name,cid,sid,founded,age
0,828837,gamy-eggplant-birman,62818,815255,06/04/2010,9
1,723295,wimpy-cerise-flamingo,82874,652943,11/24/2015,4
2,194132,gloppy-chocolate-ray,62762,810891,10/13/2011,8
3,203253,gamy-green-beetle,12878,115094,09/11/2016,3
4,305109,stealthy-teal-tapir,59628,174951,03/25/2008,11


### Coordinates

In [132]:
coordinates_df = pd.DataFrame(columns = ['id', 'latitude', 'longitude', 'city', 'country_code'])

coordinates_df['id'] = restaurant_df['cid']
coordinates_df['latitude'] = df_lat['Latitude']
coordinates_df['longitude'] = df_lat['Longitude']
coordinates_df['city'] = df_lat['City']
coordinates_df['country_code'] = df_lat['Country']

coordinates_df.head(5)

/home/eva/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,id,latitude,longitude,city,country_code
0,62818,42.483333,1.466667,aixas,ad
1,82874,42.466667,1.500000,aixirivali,ad
2,62762,42.466667,1.500000,aixirivall,ad
3,12878,42.466667,1.500000,aixirvall,ad
4,59628,42.466667,1.483333,aixovall,ad


### Speciality

In [109]:
speciality_df = pd.DataFrame(columns = [ 'id', 'name', 'calories', 'price', 'cuisine_name', 'is_vegan', 
                                        'is_gluten_free', 'is_lactose_free'])

speciality_names = [line.strip() for line in open("specialityNames.txt", 'r', encoding="utf8")]
cuisine_names = [line.strip() for line in open("cuisineNames.txt", 'r', encoding="utf8")]

speciality_df['id'] = restaurant_df['sid']
for i in range(restaurant_df.shape[0]):
    speciality_df.loc[i]['name'] = random.choice(speciality_names)
    speciality_df.loc[i]['calories'] = random.randrange(50, 1000)
    speciality_df.loc[i]['price'] = random.randrange(1, 500)
    speciality_df.loc[i]['cuisine_name'] = random.choice(cuisine_names) 
    speciality_df.loc[i]['is_vegan'] = random.randrange(100) < 25
    speciality_df.loc[i]['is_gluten_free'] =  random.randrange(100) < 15
    speciality_df.loc[i]['is_lactose_free'] = random.randrange(100) < 15

In [111]:
speciality_df.head(5)

,id,name,calories,price,cuisine_name,is_vegan,is_gluten_free,is_lactose_free
0,815255,"Veal Cutlet, Milanaise",821,394,Lebanese,False,False,True
1,652943,Whipped Cream Puffs,751,484,Chinese Islamic,True,False,False
2,810891,"""Rijs Itaafel"": Rice, Curry Sauce, Lobster, Sh...",282,73,Yamal,False,False,False
3,115094,"Prime Sirloin Steak, Maison Sandwich",342,479,Maharashtrian,False,False,False
4,174951,"An Inattention on Part of Waitress, Please Rep...",920,318,Taiwanese,False,False,False


### Reviews

In [105]:
reviews_df = pd.DataFrame(columns=['id', 'rid', 'first_Name','last_Name','rating'])

reviews_num = restaurant_df.shape[0] * 3
for i in range(reviews_num): 
    
    reviews_df.loc[i] = [np.random.randint(100000, 999999), 
                         restaurant_df['id'][random.randrange(restaurant_df.shape[0])],
                         names.get_first_name(),names.get_last_name(), np.random.randint(1,5)]

reviews_df.head(5)

,id,rid,first_Name,last_Name,rating
0,801443,493079,Carol,Henderson,1
1,278425,932382,Pauline,Tarnowski,4
2,595869,598850,Weston,Wilson,3
3,367615,693787,Stephanie,Doctor,1
4,943894,977072,Dana,Maretti,1


# Add Noise

* One case we thought for noise is that some restaurants either do not have specility dish or their speciality is missing. Thus we changed some ids from speciality so that sid from restaurants and id from speciality do not match.

In [113]:
def noise_gen(df, nn, range_, col):
    
    for i in range(nn):
        df.loc[np.random.randint(0, df.shape[0]-1)][col] = np.random.randint(range_[0], range_[1])

In [114]:
noise_gen(restaurant_df, int(restaurant_df.shape[0] * 1/100), (1000, 9000), 'id')
restaurant_df.head(5)

,id,name,cid,sid,founded,age
0,828837,gamy-eggplant-birman,62818,815255,06/04/2010,9
1,723295,wimpy-cerise-flamingo,82874,652943,11/24/2015,4
2,194132,gloppy-chocolate-ray,62762,810891,10/13/2011,8
3,203253,gamy-green-beetle,12878,115094,09/11/2016,3
4,305109,stealthy-teal-tapir,59628,174951,03/25/2008,11


* Review table contains values out of range 1-5.

In [129]:
noise_gen(reviews_df, int(reviews_df.shape[0] * 1/10), (-2, 0), 'rating')
noise_gen(reviews_df, int(reviews_df.shape[0] * 1/10), (6, 10), 'rating')
reviews_df.head(20)

,id,rid,first_Name,last_Name,rating
0,801443,493079,Carol,Henderson,1
1,278425,932382,Pauline,Tarnowski,4
2,595869,598850,Weston,Wilson,3
3,367615,693787,Stephanie,Doctor,1
4,943894,977072,Dana,Maretti,9
5,625836,335049,Kerry,Aten,2
6,882939,935095,June,Richardson,4
7,837223,300015,Jody,Howard,1
8,961026,246538,Irvin,Jackson,2
9,708525,755012,Zane,Murphy,7


* Mismatch in restaurant table between founded year and its age.

In [118]:
errors = 100
for i in range(errors):
    # throw a random number to use as index of the df
    pos = np.random.randint(0, len(restaurant_df))
    if pos%2:
        restaurant_df.loc[pos, 'age'] -= 2
    else:
        restaurant_df.loc[pos, 'age'] += 2

In [119]:
restaurant_df.head(5)

,id,name,cid,sid,founded,age
0,828837,gamy-eggplant-birman,62818,815255,06/04/2010,9
1,723295,wimpy-cerise-flamingo,82874,652943,11/24/2015,4
2,194132,gloppy-chocolate-ray,62762,810891,10/13/2011,8
3,203253,gamy-green-beetle,12878,115094,09/11/2016,3
4,305109,stealthy-teal-tapir,59628,174951,03/25/2008,11


* Null values insertion in random places in each dataframe

In [120]:
def add_missing_values(df, number, col=None):
    df_cols = list(df.columns)
    col_idx = col
    for i in range(number):
        pos = np.random.randint(0, len(df))
        if col == None:
            # mess up the whole dataframe randomly
            col_idx = df_cols[np.random.randint(2, len(df_cols))]

        df.loc[pos, col_idx] = None

In [133]:
null_rest  = int(1000)
null_coord = int(0.1 / coordinates_df.shape[0])
null_spec  = int(0.1 / speciality_df.shape[0])
null_rev   = int(0.1 / restaurant_df.shape[0])

add_missing_values(restaurant_df, null_rest)
add_missing_values(coordinates_df,null_coord)
add_missing_values(speciality_df, null_spec)
add_missing_values(reviews_df,    null_rev)

In [134]:
coordinates_df

,id,latitude,longitude,city,country_code
0,62818,42.483333,1.466667,aixas,ad
1,82874,42.466667,1.500000,aixirivali,ad
2,62762,42.466667,1.500000,aixirivall,ad
3,12878,42.466667,1.500000,aixirvall,ad
4,59628,42.466667,1.483333,aixovall,ad
5,22810,42.500000,1.516667,andorra,ad
6,66886,42.500000,1.516667,andorra la vella,ad
7,48892,42.500000,1.516667,andorra-vieille,ad
8,74728,42.500000,1.516667,andorre,ad
9,33179,42.500000,1.516667,andorre-la-vieille,ad


In [124]:
restaurant_df.head(30)

,id,name,cid,sid,founded,age
0,828837,gamy-eggplant-birman,62818,815255,06/04/2010,9
1,723295,wimpy-cerise-flamingo,82874,652943,11/24/2015,4
2,194132,gloppy-chocolate-ray,62762,810891,10/13/2011,8
3,203253,gamy-green-beetle,12878,115094,09/11/2016,3
4,305109,stealthy-teal-tapir,59628,174951,03/25/2008,11
5,934891,cheeky-aquamarine-reindeer,22810,793055,04/01/2016,3
6,881407,shaggy-lavender-alligator,66886,236874,02/01/2011,8
7,883231,sleazy-magenta-armadillo,48892,373104,07/19/2011,8
8,563774,foggy-malachite-catfish,74728,293322,09/15/2008,11
9,448930,squeaky-pumpkin-albatross,33179,563530,03/05/2010,9


* Duplicate in latitude longitude  due to different names of the same city. Already existed in the file we used.

* Outiliers for price

In [125]:
noise_gen(speciality_df, int(speciality_df.shape[0] * 1/100), (5000, 30000), 'price')
speciality_df.head(5)

,id,name,calories,price,cuisine_name,is_vegan,is_gluten_free,is_lactose_free
0,815255,"Veal Cutlet, Milanaise",821,394,Lebanese,False,False,True
1,652943,Whipped Cream Puffs,751,484,Chinese Islamic,True,False,False
2,810891,"""Rijs Itaafel"": Rice, Curry Sauce, Lobster, Sh...",282,73,Yamal,False,False,False
3,115094,"Prime Sirloin Steak, Maison Sandwich",342,479,Maharashtrian,False,False,False
4,174951,"An Inattention on Part of Waitress, Please Rep...",920,318,Taiwanese,False,False,False


In [126]:
restaurant_df.to_excel('restaurants.xlsx')
speciality_df.to_excel('speciality.xlsx')
reviews_df.to_excel('reviews.xlsx')
coordinates_df.to_excel('coordinates.xlsx')

In [130]:
reviews_df.to_excel('reviews.xlsx')

In [135]:
coordinates_df.to_excel('coordinates.xlsx')